## Implementação

In [33]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
import time

A implementação foi feita com base no código R disponibilizado pelos autores do artigo.

In [34]:
# Statistics from the styrene exposure data set
yi = np.array([3.302,4.587,5.052,5.089,4.498,5.186,4.915,4.876,5.262,5.009,5.602,4.336,4.813])
SST = 11.430
SSE = 14.711

In [35]:
#função que implementa o método de amostragem Gibbs em blocos
def BlockedGibbs(yi, m, SSE, a = -1/2, b = 0, num_it = 1000, burnout = 30):
    sigmae_estimation = []
    sigmatheta_estimation = []
    correlation = []
    mean_estimation = []
    it = 0
    
    q = len(yi)
    M = q*m
    y_bar = np.mean(yi)
    
    theta =  yi
    mu = y_bar
    eta = np.append(theta, mu)
    
    #random seed
    matrix = np.append(np.diag(np.ones(q)), -(np.ones((q,1))), axis = 1)
    mm = np.transpose(matrix) @ matrix
    
    w1 = np.transpose(eta) @ mm @ eta
    w2 = m*np.transpose(yi-theta) @ (yi-theta)
    lt = np.random.gamma(q/2 + a, scale  = 1/((w1/2)))
    le = np.random.gamma(M/2 + b, scale  = 1/(((w2 + SSE)/2)))
    
    for iterations in range(num_it+burnout):
        #eta
        t = M*lt*le/(lt+m*le)
        d = np.sqrt(lt+m*le)
        invL = np.diag(np.append(np.ones(q)/d,1/np.sqrt(t)))
        invL[-1,:-1] = lt/(d**2*np.sqrt(t))*np.ones(q)
        
        V = np.transpose(invL) @ invL
        etha0 = le*m*V @ np.append(yi, 0)
        eta = np.transpose(etha0)+np.transpose(invL) @ np.transpose(np.random.normal(size = q+1))
        
        #sigma
        w1 = np.transpose(eta) @ mm @ eta
        w2 = m*np.transpose(yi-eta[:-1]) @ (yi-eta[:-1])
        lt = np.random.gamma(q/2 + a, scale  = 1/((w1/2)))
        le = np.random.gamma(M/2 + b, scale  = 1/(((w2 + SSE)/2)))
        
        #aceite ou discarte
        it += 1
        if it >= burnout:
            se = 1/le
            st = 1/lt
            sigmae_estimation.append(se)
            sigmatheta_estimation.append(st)
            correlation.append(st/(st + se))
    
    return sigmae_estimation, sigmatheta_estimation, correlation

In [36]:
# tempo de execução
start = time.time()
sigmae_matrix, sigmatheta_matrix, corr_matrix = BlockedGibbs(yi, 3, SSE, a = -1/2, b = 0, num_it = 87169, burnout = 30)
end = time.time()
print(end-start)

8.088372468948364


In [37]:
print('sigma_theta= ', np.mean(sigmatheta_matrix)) #, CI(sigmatheta_matrix)) 
print('sigma_e= ', np.mean(sigmae_matrix)) #, CI(sigmae_matrix)) 
print('corr= ', np.mean(corr_matrix))#, CI(corr_matrix)) 

sigma_theta=  0.186341792230906
sigma_e=  0.6211184889008688
corr=  0.20880550233203934


In [ ]:
# calculo da variância assintótica


In [38]:
# MCSE
def mcse(matrix, k = 20):
    mean  = np.mean(matrix)
    n =  len(matrix)
    m = int(n/k)
    summation = 0
    
    for j in range(k):
        mean_batch = np.mean(matrix[j*m:(j+1)*m])
        summation += (mean_batch-mean)**2
    
    output = summation/(k*(k-1))
    
    return output

In [42]:
print('sigma_theta_MCSE= ', mcse(sigmatheta_matrix))
print('sigma_e_MCSE= ', mcse(sigmae_matrix))
print('corr_confidence_MCSE= ', mcse(corr_matrix))

sigma_theta_MCSE=  8.507984155875154e-06
sigma_e_MCSE=  1.9772030229086873e-06
corr_confidence_MCSE=  8.308081467350761e-06


In [43]:
# intervalo de confiança assintotico
def CI(matrix, k = 20, alpha = 0.95):
    out_in = mcse(matrix, k = k)
    return np.mean(matrix)-np.sqrt(out_in)*t.ppf(1-(1-alpha)/2, k-1), np.mean(matrix)+np.sqrt(out_in)*t.ppf(1-(1-alpha)/2, k-1)

In [44]:
print('sigma_theta_confidence_interval = ', CI(sigmatheta_matrix)) 
print('sigma_e_confidence_interval= ', CI(sigmae_matrix)) 
print('corr_confidence_interval= ', CI(corr_matrix)) 

sigma_theta_confidence_interval =  (0.18023676569753103, 0.19244681876428096)
sigma_e_confidence_interval=  (0.6181754239110522, 0.6240615538906855)
corr_confidence_interval=  (0.20277262363841697, 0.21483838102566172)


In [51]:
def ploting(matrix, skip = 1, interval = False, rang = None, subplot = False):
    means = []
    if interval == True:
        upper_bound = []
        lower_bound = []
    
    max_it = int(len(matrix)/skip)
    num = []
    for i in range(max_it):
        means.append(np.mean(matrix[:i*skip+1]))
        num.append(i*skip+1)
        
        if interval == True:
            ci = CI(matrix[:i*skip+1])
            lower_bound.append(ci[0])
            upper_bound.append(ci[1])
    
    if subplot == False:
        plt.plot(num, means)
        plt.plot(num, np.mean(matrix)*np.ones(len(means)), 'r--')

        if interval == True:
            plt.plot(num, lower_bound, linestyle = 'solid', color = 'red', linewidth = 0.5)
            plt.plot(num, upper_bound, linestyle = 'solid', color = 'red', linewidth = 0.5)
            #plt.fill_between(num, lower_bound, upper_bound, color = 'red', alpha = 0.05)
        if rang != None:
            plt.ylim(rang)
        plt.show()
    else:
        return num, means, np.mean(matrix)*np.ones(len(means))

In [ ]:
ploting(sigmatheta_matrix, skip = 100, rang = [0.1, 0.25], interval = True)

C:\Users\Julie\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Julie\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
